# Exp 18

### compare models

In [86]:
import os
import sys
import datetime
import tensorflow
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import OrderedDict

import models
import report
import block_sampler
import batch_encoder
import callbacks
from dataset import Dataset
from report import Reporter
from trainer import Trainer
from attention import Attention, showSample
from batch_encoder import Dataset
from block_sampler import count_sectors, BlockSamplerByCategory, BlockSamplerFirstBlock
from batch_encoder import xs_encoder_8bits_11, BatchEncoder
from filter_random import gen_rndchk_models, evaluate_rnd_model, filter_dataset

from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, LSTM, Dense, Activation, TimeDistributed, Flatten, Dot, Softmax, Lambda, RepeatVector, Multiply, Permute, Reshape, BatchNormalization
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from trainer import TrainResults

In [39]:
tensorflow.__version__, tensorflow.keras.__version__

('1.14.0', '2.2.4-tf')

In [40]:
tensorflow.test.is_gpu_available(), tensorflow.test.is_built_with_cuda()

(False, False)

In [70]:
# raw_dataset_folder='/home/user/sample200'
raw_dataset_folder='../datasets/govdocs1/sample200'
random_dataset_folder='../datasets/random'
minimum=200
maximum=200
result_dir = 'results/exp18'
os.makedirs(result_dir, exist_ok=True)

In [42]:
def D(classes, len_byte_vector):
    last = l0 = Input(shape=(512,len_byte_vector))
    last = Flatten()(last)
    last = Dense(classes)(last)
    last = Activation('softmax')(last)
    name = sys._getframe().f_code.co_name
    model = tf.keras.Model([l0], last, name=name)
    model.compile(loss=tf.keras.losses.categorical_crossentropy,
        optimizer=tf.keras.optimizers.Adam(),
        metrics=['binary_accuracy', 'categorical_accuracy'])
    return model

In [43]:
def LD(classes, len_byte_vector):
    last = l0 = Input(shape=(512,len_byte_vector))
    last = LSTM(32)(last)
    last = Dense(classes)(last)
    last = Activation('softmax')(last)
    name = sys._getframe().f_code.co_name
    model = tf.keras.Model([l0], last, name=name)
    model.compile(loss=tf.keras.losses.categorical_crossentropy,
        optimizer=tf.keras.optimizers.Adam(),
        metrics=['binary_accuracy', 'categorical_accuracy'])
    return model

In [107]:
def LL(classes, len_byte_vector):
    last = l0 = Input(shape=(512,len_byte_vector))
    last = LSTM(32, return_sequences=True)(last)
    last = LSTM(classes)(last)
    last = Activation('softmax')(last)
    name = sys._getframe().f_code.co_name
    model = tf.keras.Model([l0], last, name=name)
    model.compile(loss=tf.keras.losses.categorical_crossentropy,
        optimizer=tf.keras.optimizers.Adam(),
        metrics=['binary_accuracy', 'categorical_accuracy'])
    return model

In [45]:
def CD(classes, len_byte_vector):
    last = l0 = Input(shape=(512,len_byte_vector))
    last = Conv1D(64,(64,),strides=8)(last)
    last = Flatten()(last)
    last = Dense(classes)(last)
    last = Activation('softmax')(last)
    name = sys._getframe().f_code.co_name
    model = tf.keras.Model([l0], last, name=name)
    model.compile(loss=tf.keras.losses.categorical_crossentropy,
        optimizer=tf.keras.optimizers.Adam(),
        metrics=['binary_accuracy', 'categorical_accuracy'])
    return model

In [46]:
def CM(classes, len_byte_vector):
    last = l0 = Input(shape=(512,len_byte_vector))
    last = Conv1D(classes, (32,), strides=1)(last)
    last = MaxPooling1D(pool_size=481, strides=1)(last)
    last = Flatten()(last)
    last = Activation('softmax')(last)
    name = sys._getframe().f_code.co_name
    model = tf.keras.Model([l0], last, name=name)
    model.compile(loss=tf.keras.losses.categorical_crossentropy,
        optimizer=tf.keras.optimizers.Adam(),
        metrics=['binary_accuracy', 'categorical_accuracy'])
    return model

In [47]:
def CCM(classes, len_byte_vector):
    last = l0 = Input(shape=(512,len_byte_vector))
    last = Conv1D(classes, (32,), strides=1)(last)
    last = Conv1D(classes, (2,), strides=2)(last)
    last = MaxPooling1D(pool_size=240, strides=1)(last)
    last = Flatten()(last)
    last = Activation('softmax')(last)
    name = sys._getframe().f_code.co_name
    model = tf.keras.Model([l0], last, name=name)
    model.compile(loss=tf.keras.losses.categorical_crossentropy,
        optimizer=tf.keras.optimizers.Adam(),
        metrics=['binary_accuracy', 'categorical_accuracy'])
    return model

In [48]:
def CL(classes, len_byte_vector):
    last = l0 = Input(shape=(512,len_byte_vector))
    last = Conv1D(classes, (32,), strides=32)(last)
    last = LSTM(classes)(last)
    last = Activation('softmax')(last)
    name = sys._getframe().f_code.co_name
    model = tf.keras.Model([l0], last, name=name)
    model.compile(loss=tf.keras.losses.categorical_crossentropy,
        optimizer=tf.keras.optimizers.Adam(),
        metrics=['binary_accuracy', 'categorical_accuracy'])
    return model

In [49]:
def CML(classes, len_byte_vector):
    last = l0 = Input(shape=(512,len_byte_vector))
    last = Conv1D(32, (32,), strides=32)(last)
    last = MaxPooling1D(pool_size=2, strides=2, data_format='channels_first')(last)    
    last = LSTM(classes)(last)
    last = Activation('softmax')(last)
    name = sys._getframe().f_code.co_name
    model = tf.keras.Model([l0], last, name=name)
    model.compile(loss=tf.keras.losses.categorical_crossentropy,
        optimizer=tf.keras.optimizers.Adam(),
        metrics=['binary_accuracy', 'categorical_accuracy'])
    return model

In [50]:
def CLL(classes, len_byte_vector):
    last = l0 = Input(shape=(512,len_byte_vector))
    last = Conv1D(32, (32,), strides=32)(last)
    last = LSTM(64, return_sequences=True)(last)
    last = LSTM(classes)(last)
    last = Activation('softmax')(last)
    name = sys._getframe().f_code.co_name
    model = tf.keras.Model([l0], last, name=name)
    model.compile(loss=tf.keras.losses.categorical_crossentropy,
        optimizer=tf.keras.optimizers.Adam(),
        metrics=['binary_accuracy', 'categorical_accuracy'])
    return model

In [51]:
def CLD(classes, len_byte_vector):
    last = l0 = Input(shape=(512,len_byte_vector))
    last = Conv1D(256, (16,), strides=16)(last)
    last = LSTM(128)(last)
    last = Dense(classes)(last)
    last = Activation('softmax')(last)
    name = sys._getframe().f_code.co_name
    model = tf.keras.Model([l0], last, name=name)
    model.compile(loss=tf.keras.losses.categorical_crossentropy,
        optimizer=tf.keras.optimizers.Adam(),
        metrics=['binary_accuracy', 'categorical_accuracy'])
    return model

In [52]:
def CCL(classes, len_byte_vector):
    last = l0 = Input(shape=(512,len_byte_vector))
    last = Conv1D(128, (8,), strides=8)(last)
    last = Conv1D(64, (8,), strides=8)(last)
    last = LSTM(classes)(last)
    last = Activation('softmax')(last)
    name = sys._getframe().f_code.co_name
    model = tf.keras.Model([l0], last, name=name)
    model.compile(loss=tf.keras.losses.categorical_crossentropy,
        optimizer=tf.keras.optimizers.Adam(),
        metrics=['binary_accuracy', 'categorical_accuracy'])
    return model

In [53]:
def CCLL(classes, len_byte_vector):
    last = l0 = Input(shape=(512,len_byte_vector))
    last = Conv1D(128, (8,), strides=8)(last)
    last = Conv1D(64, (8,), strides=8)(last)
    last = LSTM(64, return_sequences=True)(last)
    last = LSTM(classes)(last)
    last = Activation('softmax')(last)
    name = sys._getframe().f_code.co_name
    model = tf.keras.Model([l0], last, name=name)
    model.compile(loss=tf.keras.losses.categorical_crossentropy,
        optimizer=tf.keras.optimizers.Adam(),
        metrics=['binary_accuracy', 'categorical_accuracy'])
    return model

In [54]:
def CMCML(classes, len_byte_vector):
    last = l0 = Input(shape=(512,len_byte_vector))
    last = Conv1D(128, (8,), strides=8)(last)
    last = MaxPooling1D(pool_size=2, strides=2, data_format='channels_first')(last)
    last = Conv1D(64, (8,), strides=8)(last)
    last = MaxPooling1D(pool_size=2, strides=2, data_format='channels_first')(last)
    last = LSTM(classes)(last)
    last = Activation('softmax')(last)
    name = sys._getframe().f_code.co_name
    model = tf.keras.Model([l0], last, name=name)
    model.compile(loss=tf.keras.losses.categorical_crossentropy,
        optimizer=tf.keras.optimizers.Adam(),
        metrics=['binary_accuracy', 'categorical_accuracy'])
    return model

In [55]:
def CMCMLL(classes, len_byte_vector):
    last = l0 = Input(shape=(512,len_byte_vector))
    last = Conv1D(128, (8,), strides=8)(last)
    last = MaxPooling1D(pool_size=2, strides=2, data_format='channels_first')(last)
    last = Conv1D(64, (8,), strides=8)(last)
    last = MaxPooling1D(pool_size=2, strides=2, data_format='channels_first')(last)
    last = LSTM(64, return_sequences=True)(last)
    last = LSTM(classes)(last)
    last = Activation('softmax')(last)
    name = sys._getframe().f_code.co_name
    model = tf.keras.Model([l0], last, name=name)
    model.compile(loss=tf.keras.losses.categorical_crossentropy,
        optimizer=tf.keras.optimizers.Adam(),
        metrics=['binary_accuracy', 'categorical_accuracy'])
    return model

In [110]:
classes = 30
len_byte_vector=256
mymodels = [
    D(classes, len_byte_vector),

    LD(classes, len_byte_vector),
    LL(classes, len_byte_vector),

    CD(classes, len_byte_vector),
    CM(classes, len_byte_vector),
    CCM(classes, len_byte_vector),
    
    CL(classes, len_byte_vector),
    CML(classes, len_byte_vector),
    CLL(classes, len_byte_vector),
    CLD(classes, len_byte_vector),
    
    CCL(classes, len_byte_vector),
    CCLL(classes, len_byte_vector),
    CMCML(classes, len_byte_vector),
    CMCMLL(classes, len_byte_vector),
]

In [78]:
rawset = Dataset.new_from_folders(raw_dataset_folder).filter_min_max(minimum, maximum)

In [79]:
tset, vset = rawset.rnd_split_fraction_by_category(0.5)

In [111]:
class MyTrainer:
    def __init__(self,
                 model,
                 group_by='by_file',
                 xs_encoder='8bits_11',
                 validation_steps=10,
                 steps_per_epoch=10,
                 epochs=10000000,
                 max_seconds=10*60,
                 batch_size=400,
                 min_delta=1e-03,
                 patience=4,
                 blockSampler=block_sampler.BlockSampler,
                 batchEncoder=batch_encoder.BatchEncoder):
        self.model = model
        self.group_by = group_by
        self.xs_encoder = xs_encoder
        self.validation_steps = validation_steps
        self.steps_per_epoch = steps_per_epoch
        self.epochs = epochs
        self.max_seconds = max_seconds
        self.batch_size = batch_size
        self.min_delta = min_delta
        self.patience = patience
        self.blockSampler = blockSampler
        self.batchEncoder = batchEncoder

    def train(self, tset, vset):
        tsampler = self.blockSampler(tset, group_by=self.group_by)
        tbenc = self.batchEncoder(tsampler, self.batch_size,
                                  xs_encoder=self.xs_encoder)

        vsampler = self.blockSampler(vset, group_by=self.group_by)
        vbenc = self.batchEncoder(vsampler, self.batch_size,
                                  xs_encoder=self.xs_encoder)

        model = self.model

        timeIt = callbacks.TimeIt()

        history = model.fit_generator(iter(tbenc),
                                      validation_data=iter(vbenc),
                                      validation_steps=self.validation_steps,
                                      steps_per_epoch=self.steps_per_epoch,
                                      epochs=self.epochs,
                                      verbose=1,
                                      callbacks=[
            timeIt,
            # callbacks.SaveModel(os.path.join(result_dir, model.name + '.h5')),
            callbacks.TimeLimit(self.max_seconds),
#             EarlyStopping(monitor='val_categorical_accuracy',
#                           min_delta=self.min_delta, patience=self.patience),
            TensorBoard(
                log_dir=os.path.join(result_dir, model.name),
            ),
        ],
            use_multiprocessing=False,
            workers=0,
        )
        return TrainResults(
            model=model,
            history=history,
            metrics=['val_binary_accuracy', 'val_categorical_accuracy'],
            elapsed=timeIt.elapsed,
        )

In [114]:
r = report.Reporter2(result_dir + '/models.tsv')
for model in mymodels:
    print(model.name)
    result = MyTrainer(
        model,
        xs_encoder='one_hot',
        batch_size=100,
        steps_per_epoch=30,
        validation_steps=30
    ).train(tset, vset)
    print(result.elapsed)
    r.line(name=result.model.name,
           elapsed=result.elapsed,
           **report.report_epochs(**result._asdict()),
           **report.report_metrics(result.history, result.metrics),
          )

r.close()

D
Epoch 1/10000000
30/30 [==============================] - 19s 641ms/step - loss: 2.7989 - binary_accuracy: 0.9674 - categorical_accuracy: 0.2287 - val_loss: 2.4230 - val_binary_accuracy: 0.9682 - val_categorical_accuracy: 0.3097
Epoch 2/10000000
30/30 [==============================] - 14s 463ms/step - loss: 2.1586 - binary_accuracy: 0.9706 - categorical_accuracy: 0.3670 - val_loss: 2.1724 - val_binary_accuracy: 0.9701 - val_categorical_accuracy: 0.3693
Epoch 3/10000000
30/30 [==============================] - 14s 465ms/step - loss: 1.9295 - binary_accuracy: 0.9724 - categorical_accuracy: 0.4343 - val_loss: 2.0879 - val_binary_accuracy: 0.9706 - val_categorical_accuracy: 0.3780
Epoch 4/10000000
30/30 [==============================] - 14s 464ms/step - loss: 1.8146 - binary_accuracy: 0.9739 - categorical_accuracy: 0.4587 - val_loss: 2.0141 - val_binary_accuracy: 0.9716 - val_categorical_accuracy: 0.4033
Epoch 5/10000000
30/30 [==============================] - 14s 456ms/step - loss: 1

Epoch 37/10000000
30/30 [==============================] - 14s 460ms/step - loss: 1.1579 - binary_accuracy: 0.9834 - categorical_accuracy: 0.6487 - val_loss: 1.8561 - val_binary_accuracy: 0.9741 - val_categorical_accuracy: 0.4537
Epoch 38/10000000
30/30 [==============================] - 14s 457ms/step - loss: 1.1317 - binary_accuracy: 0.9842 - categorical_accuracy: 0.6593 - val_loss: 1.8806 - val_binary_accuracy: 0.9740 - val_categorical_accuracy: 0.4450
Epoch 39/10000000
30/30 [==============================] - 14s 468ms/step - loss: 1.1219 - binary_accuracy: 0.9841 - categorical_accuracy: 0.6633 - val_loss: 1.8683 - val_binary_accuracy: 0.9739 - val_categorical_accuracy: 0.4477
Epoch 40/10000000
30/30 [==============================] - 14s 463ms/step - loss: 1.1291 - binary_accuracy: 0.9840 - categorical_accuracy: 0.6580 - val_loss: 1.9068 - val_binary_accuracy: 0.9737 - val_categorical_accuracy: 0.4393
Epoch 41/10000000
30/30 [==============================] - 14s 461ms/step - loss

Epoch 10/10000000
30/30 [==============================] - 46s 2s/step - loss: 2.9853 - binary_accuracy: 0.9667 - categorical_accuracy: 0.0647 - val_loss: 2.9661 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.0650
Epoch 11/10000000
30/30 [==============================] - 46s 2s/step - loss: 2.9390 - binary_accuracy: 0.9667 - categorical_accuracy: 0.0837 - val_loss: 2.9350 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.0870
Epoch 12/10000000
30/30 [==============================] - 46s 2s/step - loss: 2.9204 - binary_accuracy: 0.9667 - categorical_accuracy: 0.0930 - val_loss: 2.9232 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.0927
Epoch 13/10000000
30/30 [==============================] - 46s 2s/step - loss: 2.9071 - binary_accuracy: 0.9667 - categorical_accuracy: 0.0917 - val_loss: 2.9029 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.0943
610.1934232711792
CD
Epoch 1/10000000
30/30 [==============================] - 28s 931ms/ste

30/30 [==============================] - 38s 1s/step - loss: 2.8886 - binary_accuracy: 0.9667 - categorical_accuracy: 0.3273 - val_loss: 2.8462 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.3113
Epoch 4/10000000
30/30 [==============================] - 38s 1s/step - loss: 2.7356 - binary_accuracy: 0.9667 - categorical_accuracy: 0.3550 - val_loss: 2.7196 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.3310
Epoch 5/10000000
30/30 [==============================] - 39s 1s/step - loss: 2.6072 - binary_accuracy: 0.9667 - categorical_accuracy: 0.3827 - val_loss: 2.6096 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.3427
Epoch 6/10000000
30/30 [==============================] - 38s 1s/step - loss: 2.4932 - binary_accuracy: 0.9668 - categorical_accuracy: 0.4033 - val_loss: 2.5142 - val_binary_accuracy: 0.9668 - val_categorical_accuracy: 0.3753
Epoch 7/10000000
30/30 [==============================] - 39s 1s/step - loss: 2.3983 - binary_accuracy: 0.9670 - 

Epoch 7/10000000
30/30 [==============================] - 14s 469ms/step - loss: 2.5418 - binary_accuracy: 0.9667 - categorical_accuracy: 0.2460 - val_loss: 2.5586 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.2450
Epoch 8/10000000
30/30 [==============================] - 14s 466ms/step - loss: 2.5104 - binary_accuracy: 0.9667 - categorical_accuracy: 0.2653 - val_loss: 2.5330 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.2490
Epoch 9/10000000
30/30 [==============================] - 14s 466ms/step - loss: 2.4878 - binary_accuracy: 0.9667 - categorical_accuracy: 0.2570 - val_loss: 2.4965 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.2583
Epoch 10/10000000
30/30 [==============================] - 14s 452ms/step - loss: 2.4646 - binary_accuracy: 0.9667 - categorical_accuracy: 0.2707 - val_loss: 2.5021 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.2743
Epoch 11/10000000
30/30 [==============================] - 13s 447ms/step - loss: 2

...

Epoch 19/10000000
30/30 [==============================] - 14s 450ms/step - loss: 2.3768 - binary_accuracy: 0.9667 - categorical_accuracy: 0.3370 - val_loss: 2.4181 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.3370
Epoch 20/10000000
30/30 [==============================] - 13s 449ms/step - loss: 2.3632 - binary_accuracy: 0.9667 - categorical_accuracy: 0.3360 - val_loss: 2.4059 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.3287
Epoch 21/10000000
30/30 [==============================] - 14s 453ms/step - loss: 2.3552 - binary_accuracy: 0.9667 - categorical_accuracy: 0.3533 - val_loss: 2.4048 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.3170
Epoch 22/10000000
30/30 [==============================] - 14s 452ms/step - loss: 2.3538 - binary_accuracy: 0.9667 - categorical_accuracy: 0.3503 - val_loss: 2.4255 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.3167
Epoch 23/10000000
30/30 [==============================] - 13s 445ms/step - loss

30/30 [==============================] - 14s 461ms/step - loss: 2.4566 - binary_accuracy: 0.9667 - categorical_accuracy: 0.2880 - val_loss: 2.4796 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.2947
Epoch 11/10000000
30/30 [==============================] - 14s 458ms/step - loss: 2.4387 - binary_accuracy: 0.9667 - categorical_accuracy: 0.2847 - val_loss: 2.4668 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.2930
Epoch 12/10000000
30/30 [==============================] - 14s 454ms/step - loss: 2.4239 - binary_accuracy: 0.9667 - categorical_accuracy: 0.3097 - val_loss: 2.4371 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.2890
Epoch 13/10000000
30/30 [==============================] - 14s 454ms/step - loss: 2.4084 - binary_accuracy: 0.9667 - categorical_accuracy: 0.3007 - val_loss: 2.4474 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.3013
Epoch 14/10000000
30/30 [==============================] - 14s 457ms/step - loss: 2.4086 - binary_

Epoch 3/10000000
30/30 [==============================] - 13s 440ms/step - loss: 2.8102 - binary_accuracy: 0.9667 - categorical_accuracy: 0.1880 - val_loss: 2.7380 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.2013
Epoch 4/10000000
30/30 [==============================] - 13s 446ms/step - loss: 2.6640 - binary_accuracy: 0.9667 - categorical_accuracy: 0.1957 - val_loss: 2.6069 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.2157
Epoch 5/10000000
30/30 [==============================] - 13s 449ms/step - loss: 2.5970 - binary_accuracy: 0.9667 - categorical_accuracy: 0.2173 - val_loss: 2.5898 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.2220
Epoch 6/10000000
30/30 [==============================] - 13s 444ms/step - loss: 2.5540 - binary_accuracy: 0.9667 - categorical_accuracy: 0.2630 - val_loss: 2.5563 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.2743
Epoch 7/10000000
30/30 [==============================] - 13s 443ms/step - loss: 2.5

Epoch 39/10000000
30/30 [==============================] - 14s 460ms/step - loss: 2.2781 - binary_accuracy: 0.9667 - categorical_accuracy: 0.4090 - val_loss: 2.3434 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.3893
Epoch 40/10000000
30/30 [==============================] - 14s 459ms/step - loss: 2.2819 - binary_accuracy: 0.9667 - categorical_accuracy: 0.4010 - val_loss: 2.3516 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.3837
Epoch 41/10000000
30/30 [==============================] - 14s 462ms/step - loss: 2.2788 - binary_accuracy: 0.9667 - categorical_accuracy: 0.4237 - val_loss: 2.3457 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.3733
Epoch 42/10000000
30/30 [==============================] - 14s 461ms/step - loss: 2.2738 - binary_accuracy: 0.9667 - categorical_accuracy: 0.4217 - val_loss: 2.3529 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.3917
Epoch 43/10000000
30/30 [==============================] - 14s 457ms/step - loss

30/30 [==============================] - 14s 464ms/step - loss: 2.3367 - binary_accuracy: 0.9667 - categorical_accuracy: 0.3577 - val_loss: 2.4025 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.3443
Epoch 31/10000000
30/30 [==============================] - 14s 470ms/step - loss: 2.3541 - binary_accuracy: 0.9667 - categorical_accuracy: 0.3517 - val_loss: 2.4103 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.3300
Epoch 32/10000000
30/30 [==============================] - 14s 465ms/step - loss: 2.3473 - binary_accuracy: 0.9667 - categorical_accuracy: 0.3430 - val_loss: 2.3908 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.3147
Epoch 33/10000000
30/30 [==============================] - 14s 464ms/step - loss: 2.3238 - binary_accuracy: 0.9667 - categorical_accuracy: 0.3623 - val_loss: 2.3768 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.3430
Epoch 34/10000000
30/30 [==============================] - 14s 464ms/step - loss: 2.3283 - binary_

In [106]:
len(rawset.categories)

30